In [ ]:
#r "BoSSSpad.dll"
// #r ".\binaries\XNSEC.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

In [ ]:
using BoSSS.Application.XNSEC;

In [ ]:
BoSSSshell.WorkflowMgm.Init("CounterFlowFlame_MF_FullComparison");

In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("CounterFlowFlame_MF_FullComparison");
// var myDb = OpenOrCreateDatabase(@"C:\Databases\CounterFlowFlame");


In [ ]:
int counter = 0;
int[] mults = new int[] { 2, 5 , 11 };
Dictionary<int,Plot2Ddata[,] > PlotTableS = new Dictionary<int, Plot2Ddata[,]>();

foreach (int mult in mults) {
    counter++;

    var sess = myDb.Sessions.Where(sess => sess.Name.Contains("mult" + mult) && sess.Name.Contains("Full")).FirstOrDefault(); // Search the actual session
    if(sess == null)
        continue;
        
    var timestep_FullChem = sess.Timesteps.Last();
    double eps = 1e-4;
    double xleft = 0 + eps;
    double xright = 1.0 - eps;
    int nCells = 80; // Number of points to be picked
    double[] _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
    string[] varnames = new string[] { "VelocityX", "Temperature", "MassFraction0", "MassFraction1", "MassFraction2", "MassFraction3" };

    Dictionary<string, double[]> Results_FullComb = new Dictionary<string, double[]>();
    MultidimensionalArray[] Results_FullComb2 = new MultidimensionalArray[varnames.Length];
    Dictionary<string, double> ReferenceValuesDict = new Dictionary<string, double>();// Re-dimenzionalise variables

    ReferenceValuesDict.Add("VelocityX", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
    ReferenceValuesDict.Add("Temperature", Convert.ToDouble(sess.KeysAndQueries["TRef"]));
    ReferenceValuesDict.Add("MassFraction0", 1.0);
    ReferenceValuesDict.Add("MassFraction1", 1.0);
    ReferenceValuesDict.Add("MassFraction2", 1.0);
    ReferenceValuesDict.Add("MassFraction3", 1.0);
    ReferenceValuesDict.Add("Length", Convert.ToDouble(sess.KeysAndQueries["LRef"]));

    
    //Pick and save results from the BoSSS calculation 
    int cnt = -1;
    double[] _xNodesDim = new double[_xNodes.Length];
    foreach (var varname in varnames) {
        cnt++;
        double[] Yvalues = new double[nCells + 1];
        var field = timestep_FullChem.Fields.Where(Fi => (Fi.Identification == varname)).Last();
        for (int n = 0; n < _xNodes.Length; n++) {
            _xNodesDim[n] = _xNodes[n] * ReferenceValuesDict["Length"]; // Redimensionalise the x coordinate
            double pickedVal = field.ProbeAt(new double[] { _xNodes[n], 0.00000001 });    // Pick value along the central axis        
            Yvalues[n] = pickedVal * ReferenceValuesDict[varname];             // Re.dimenzionalise varliables and store it in array
        }
        var xyVals = MultidimensionalArray.Create(_xNodes.Length, _xNodes.Length);
        for (int i = 0; i < _xNodesDim.Length; i++) {
            xyVals[i, 0] = _xNodesDim[i];
            xyVals[i, 1] = Yvalues[i];
        }
        Results_FullComb2[cnt] = xyVals;
    }


    // Read results from the Matlab calculation of the 1D problem.
    string[] VariablesForPlot = new string[] { "VelocityX", "Temperature", "MF0", "MF1", "MF2", "MF3" };
    MultidimensionalArray[] ReferenceData = new MultidimensionalArray[VariablesForPlot.Length];
    string CurrentDocDir = Directory.GetCurrentDirectory();
    //Load data from Matlab
    for (int i = 0; i < VariablesForPlot.Length; i++) {
        string path = String.Concat(CurrentDocDir, @"\ML" + mult + "_", VariablesForPlot[i], ".txt");
        ReferenceData[i] = IMatrixExtensions.LoadFromTextFile(path);
    }

    

    Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 3];
    int cnt2 = -1;
    for (int iCol = 0; iCol < 3; iCol++) {
        for (int iRow = 0; iRow < 2; iRow++) {
            var plot = new Plot2Ddata();
            var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
            cnt2++;
            // BoSSS data
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.DashType = DashTypes.Solid;
            
            plot.AddDataGroup( "BoSSS",
                Results_FullComb2[cnt2].ExtractSubArrayShallow(-1, 0).To1DArray(),
                Results_FullComb2[cnt2].ExtractSubArrayShallow(-1, 1).To1DArray(),
                fmt);

            // Matlab data
            fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.DashType = DashTypes.Dashed;
            plot.AddDataGroup( "Matlab",
            ReferenceData[cnt2].ExtractSubArrayShallow(-1, 0).To1DArray(),
            ReferenceData[cnt2].ExtractSubArrayShallow(-1, 1).To1DArray(),
            fmt);
            plot.Title = varnames[cnt2];
            plot.TitleFont = 20;
            plot.Xlabel = "x (m)";
            
            if(iRow == 0 && iCol == 0)
                plot.Ylabel = "velocity (m/s)";
            if(iRow == 1 && iCol == 0)
                plot.Ylabel = "Temperature (K)";
            PlotTable[iRow, iCol] = plot;
        }
    }
    PlotTableS.Add(counter,PlotTable);

}

## Results for low inlet velocity case

In [ ]:
var gp = PlotTableS[1].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)

## Results for mid inlet velocity case

In [ ]:
var gp = PlotTableS[2].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)

## Results for high inlet velocity case

In [ ]:
var gp = PlotTableS[3].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)